In [ ]:
!pip install transformers datasets accelerate scikit-learn torch

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    DistilBertTokenizer,
    DistilBertForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
train_df = pd.read_csv("/content/sample_data/train.csv")
test_df = pd.read_csv("/content/sample_data/test.csv")
train_df = train_df.rename(columns={"target": "label"})
test_df = test_df.rename(columns={"target": "label"})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
train_val_split = train_dataset.train_test_split(test_size=0.2)
train_split = train_val_split["train"]
val_split = train_val_split["test"]

In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_tokenized = train_split.map(tokenize_function, batched=True)
val_tokenized = val_split.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)
train_tokenized = train_tokenized.remove_columns(["text", "keyword", "location", "id"])
val_tokenized = val_tokenized.remove_columns(["text", "keyword", "location", "id"])
test_tokenized = test_tokenized.remove_columns(["text", "keyword", "location", "id"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/6090 [00:00<?, ? examples/s]

Map:   0%|          | 0/1523 [00:00<?, ? examples/s]

Map:   0%|          | 0/3263 [00:00<?, ? examples/s]

In [ ]:
print(train_tokenized[0].keys())

dict_keys(['label', 'input_ids', 'attention_mask'])


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions)
    }

training_args = TrainingArguments(
    output_dir="./distilbert_distress",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",  # disable wandb
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    compute_metrics=compute_metrics
)

In [ ]:
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.455400,0.388931,0.850295,0.812500
2,0.298300,0.427687,0.851609,0.815359
3,0.186000,0.526442,0.838477,0.802885


TrainOutput(global_step=1143, training_loss=0.3265926256997483, metrics={'train_runtime': 239.3259, 'train_samples_per_second': 76.339, 'train_steps_per_second': 4.776, 'total_flos': 605044843361280.0, 'train_loss': 0.3265926256997483, 'epoch': 3.0})

In [ ]:
print("Predicting on test set...")
predictions = trainer.predict(test_tokenized)
pred_labels = predictions.predictions.argmax(axis=1)

Predicting on test set...


In [ ]:
for i in range(10):
    print(f"Tweet: {test_df.iloc[i]['text']}")
    print(f"Prediction: {'Distressed' if pred_labels[i] == 1 else 'Not Distressed'}\n")

Tweet: Just happened a terrible car crash
Prediction: Distressed

Tweet: Heard about #earthquake is different cities, stay safe everyone.
Prediction: Distressed

Tweet: there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all
Prediction: Distressed

Tweet: Apocalypse lighting. #Spokane #wildfires
Prediction: Distressed

Tweet: Typhoon Soudelor kills 28 in China and Taiwan
Prediction: Distressed

Tweet: We're shaking...It's an earthquake
Prediction: Distressed

Tweet: They'd probably still show more life than Arsenal did yesterday, eh? EH?
Prediction: Not Distressed

Tweet: Hey! How are you?
Prediction: Not Distressed

Tweet: What a nice hat?
Prediction: Not Distressed

Tweet: Fuck off!
Prediction: Not Distressed



In [ ]:
trainer.save_model("./my_distress_detector")
tokenizer.save_pretrained("./my_distress_detector")

('./my_distress_detector/tokenizer_config.json',
 './my_distress_detector/special_tokens_map.json',
 './my_distress_detector/vocab.txt',
 './my_distress_detector/added_tokens.json')

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("./my_distress_detector")
tokenizer = DistilBertTokenizer.from_pretrained("./my_distress_detector")